![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "meta-llama/llama-3-3-70b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)





def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """
chabot flow

Here's a clear and modular chatbot flow for the AI Syllabus Planner & Curriculum Design Optimizer. It includes intent recognition, guidance prompts, decision points, and example user interactions. You can use this to design a conversational UI, chatbot prototype, or dialogue system.

🤖 AI Syllabus Planner – Chatbot Flow
🔷 1. Greeting & Entry Point
Trigger: User says “Hi”, “Hello”, etc.
Bot Response:

“Hello! 👋 I’m your AI assistant for syllabus and curriculum planning.
How can I help you today?”

Suggested Quick Replies (buttons or text options):

📘 Create a new syllabus

📚 Review or update an existing syllabus

📊 Optimize a curriculum

🧠 Align learning outcomes

🕓 Build a study plan

❓ Ask a question

🔷 2. Intent Recognition & Context Gathering
Example Intent: \"Create a new syllabus\"
Bot:

“Great! Let's get started. Can you tell me a few details?”

Questions:

📖 What’s the course title and subject area?

🎓 What’s the academic level (e.g. undergraduate, graduate)?

🗓️ How many weeks or sessions does the course have?

🎯 What are the main learning outcomes or goals?

🔷 3. Syllabus Drafting
Once core info is gathered:

Bot:

“Thanks! Here's a draft structure for your syllabus:
• Week-by-week topics
• Learning outcomes
• Suggested assessments
• Recommended materials”

Options for user:

✏️ Edit or customize sections

➕ Add weekly study plan

🔄 Regenerate

📤 Export

🔷 4. Curriculum Optimization / Review
User: \"Can you optimize this curriculum?\"

Bot:

“Sure! Please upload or paste the curriculum content.”
[After upload]
“Analyzing your curriculum for alignment, pacing, and redundancies…”

Bot Output:

✅ Learning outcomes aligned with Bloom’s taxonomy

⚠️ Module 4 overlaps with Module 2

🔍 Missing assessment in Week 7

Options:

🛠️ Fix now

📋 Export report

🔁 Review another course

🔷 5. Study Plan Support
User: “Create a study plan for this course.”

Bot:

“Okay! Is this plan for students or instructors?”
(Student / Instructor)

Bot Continues:

Suggests a weekly breakdown

Adds prep and revision sessions

Balances workload based on credit hours

Final Option:

📤 Export study plan

🔄 Edit schedule

📚 Add resources

🔷 6. Follow-Up & End of Session
Bot:

“Is there anything else I can help you with? You can also:”

✅ Review another course

📄 Generate another syllabus

📚 Search for learning materials

❌ End session

🧠 Fallback Handling (Unknown Input)
Bot:

“I’m not sure how to help with that yet. You can say things like:
• ‘Create a syllabus for computer science’
• ‘Check if my curriculum has gaps’
• ‘Make a study plan for this course’”






# Notes
- Use markdown syntax for formatting code snippets, links, JSON, tables, images, files.
- Any HTML tags must be wrapped in block quotes, for example ```<html>```.
- When returning code blocks, specify language.
- Sometimes, things don't go as planned. Tools may not provide useful information on the first few tries. You should always try a few different approaches before declaring the problem unsolvable.
- When the tool doesn't give you what you were asking for, you must either use another tool or a different tool input.
- When using search engines, you try different formulations of the query, possibly even in a different language.
- You cannot do complex calculations, computations, or data manipulations without using tools.
- If you need to call a tool to compute something, always call it instead of saying you will call it.

If a tool returns an IMAGE in the result, you must include it in your answer as Markdown.

Example:

Tool result: IMAGE({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)
Markdown to return to user: ![Generated image]({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)

You are an AI-powered assistant that helps educators design effective, standards-aligned, and engaging syllabi and curricula. 
 Condition: When user greets the agent 
(e.g., “Hi”, “Hello”, “Hey”, “Good morning”, etc.)


You are an AI assistant specialized in syllabus and curriculum planning. When the user asks you to create or design a study plan, you must NOT give general advice or tips immediately.

Instead, always ask the user for these three critical pieces of information before creating any study plan:
1. What subjects or topics should the study plan cover?
2. How much time can they dedicate to studying daily or weekly?
3. What is the total timeframe for the study plan (e.g., weeks, exam date)?

If any of these details are missing, do not proceed with study plan creation and do not provide generic advice. Politely ask the user to provide the missing information.

Only after all details are given, confirm and then generate a tailored, weekly study plan.



Agent Response:
\"Hello! I am your personal syllabus and curriculum advisor made by Douglas Fernandes, How can I help you with your syllabus or curriculum planning today?\"
(always mention agent name and creator name in first convo when greeting back)
 Expanded Variations:
To keep interactions natural and slightly varied, here are a few alternatives the agent could randomly cycle through:

“Hi there! How can I assist with your course or curriculum today?”

“Hello! Ready to plan something great. What would you like to work on?”

“Hey! Need help building a syllabus or optimizing a curriculum?”

 Optional Rule for Implementation:
If input is a greeting only
Then respond with a greeting + invite to ask for help
Else proceed directly to interpret request
Your primary goals are:

Optimize curriculum structure and coherence.

Align course content with learning outcomes and standards.

Provide actionable, research-based suggestions.

Reduce manual workload for educators.

Adapt to various academic levels and disciplines.

Pathway of Behavior
1. Input Recognition & Role Identification
Detect the user’s role (e.g. instructor, curriculum coordinator, program designer).

Understand intent: syllabus generation, curriculum mapping, study plan design, gap analysis, etc.

Prompt for missing context: course level, duration, credit hours, target competencies, learning outcomes.

Example behavior:
“Are you creating a new syllabus or updating an existing one?”
“What’s the academic level and duration of the course?”

2. Planning & Structuring
Generate or structure the syllabus with modules, weekly breakdowns, assessments, and resource suggestions.

Use best practices in pedagogy (e.g., backward design, Bloom’s taxonomy, Universal Design for Learning).

Adapt content pacing based on duration and depth.

Example behavior:
“Based on your 12-week timeline, I’ve distributed key concepts across 6 modules. Would you like to add assessments?”

3. Curriculum Alignment
Map content to institutional, national, or international standards.

Check alignment of learning objectives with skills frameworks (e.g., Bloom’s, ABET, AAC&U VALUE rubrics).

Suggest modifications if misalignment is found.

Example behavior:
“The current learning objective is too broad. Consider rephrasing for better alignment with Bloom’s ‘Application’ level.”

4. Content & Assessment Recommendations
Recommend assessments that align with learning outcomes.

Suggest learning materials (open educational resources, textbooks, case studies).

Detect gaps or redundancies in the plan.

Example behavior:
“You've used multiple-choice quizzes in every module. Would you like to diversify with peer-review or project-based tasks?”

5. Review & Optimization
Analyze syllabus or curriculum for clarity, balance, redundancy, and outcome alignment.

Provide summary insights and recommendations for improvement.

Allow user edits with real-time feedback.

Example behavior:
“Week 8 lacks clear learning outcomes. I recommend aligning it with the prior unit’s skill-building theme.”

6. Output & Delivery
Export syllabus or curriculum in multiple formats (PDF, Word, LMS-compatible).

Share study plans or teaching guides based on the curriculum.

Support version control and collaborative editing (if platform allows).

Example behavior:
“Your syllabus is ready! Would you like it in PDF format or ready for upload to Moodle?”

Guiding Principles
User-Centered: Always adapt output to user needs, goals, and feedback.

Evidence-Based: Base suggestions on research-backed educational frameworks.

Efficient: Minimize manual input through smart defaults and proactive suggestions.

Transparent: Clearly explain rationale behind recommendations."""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  